<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>


# Gestion des tests d'intégration continue

**La liste des notebooks a éxécuter** et de leurs paramètres (override) est définie dans un **profile**.  
 - Un profile est un fichier yaml
 - Un profile par défaut est généré en (1).

Lorqu'un **profile** est prêt, il peut être soumis pour **éxécution** (2)
 - Pour chaque notebook, les paramètres sont positionnés (variables d'env.)
 - Le notebook est lu, puis éxécuté
 - Après éxécution, le notebook est sauvegardé sous son nom moodifié (output_tag)

Un **rapport d'éxécution** est généré durant l'éxécution des tests : [./logs/ci_report.json](./logs/ci_report.json)  
A la fin des tests, ce rapport peut être visualisé via le notebook : [./03-ci_report.ipynb](./03%20-%20ci_report.ipynb)


In [1]:
import cookci

## Get and Save default profile
Génère un profile par défaut comprenant tous les notebooks du moment...  
...en tout cas présent dans catalog.json ;-)

In [2]:
profile = cookci.get_ci_profile()
cookci.save_profile(profile, './ci/default.yml')

Catalog saved as ./ci/default.yml
Entries :  36


## Load a saved profile

In [3]:
smart_profile = cookci.load_profile('./ci/smart.yml')

./ci/smart.yml loaded.
Entries :  3


## Run this profile

In [4]:
cookci.reset_ci_report()
cookci.run_profile(smart_profile)

Finished file has been reset.


Notebook : LINR1
    Run notebook.....done.
    Duration :  0:00:04
    Saved as :  01-Linear-Regression==done==.ipynb

Notebook : PER57
    Run notebook.....done.
    Duration :  0:00:02
    Saved as :  01-Simple-Perceptron==done==.ipynb

Notebook : GTSRB2
    set overrides :
       FIDLE_OVERRIDE_GTSRB2_run_dir          = ./run/GTSRB2_ci
       FIDLE_OVERRIDE_GTSRB2_dataset_name     = set-24x24-L
       FIDLE_OVERRIDE_GTSRB2_batch_size       = 32
       FIDLE_OVERRIDE_GTSRB2_epochs           = 3
       FIDLE_OVERRIDE_GTSRB2_scale            = 0.1
    Run notebook...
    ************************************************************
    ** AAARG.. An error occured :  CellExecutionError
    ** See notebook :  02-First-convolutions==ERROR==.ipynb for details.
    ************************************************************
    Duration :  0:00:02
    Saved as :  02-First-convolutions==ERROR==.ipynb


---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>